In [8]:
from landlab import RasterModelGrid
from landlab.components import GroundwaterDupuitPercolator
from landlab.components import FlowAccumulator
import numpy as np

In [9]:
#initialize the grid and component

In [10]:
grid = RasterModelGrid((10, 10), xy_spacing=10.0)
elev = grid.add_zeros("topographic__elevation", at="node")
abe = grid.add_zeros("aquifer_base__elevation", at="node")
elev[:] = 5.0
gdp = GroundwaterDupuitPercolator(grid)

In [11]:
dt = 1E4
for i in range(100):
    gdp.run_one_step(dt)



In [12]:
#Set boundary conditions and initialize grid
grid = RasterModelGrid((5, 41), xy_spacing=10.0)
grid.set_closed_boundaries_at_grid_edges(True, True, False, True)

#Make a sloping, 3 m thick aquifer, initially fully saturated
elev = grid.add_zeros("topographic__elevation", at="node")
elev[:] = grid.x_of_node/100+3
base = grid.add_zeros("aquifer_base__elevation", at="node")
base[:] = grid.x_of_node/100
wt = grid.add_zeros("water_table__elevation", at="node")
wt[:] = grid.x_of_node/100 + 3

In [13]:
#initialize components
gdp = GroundwaterDupuitPercolator(grid, recharge_rate=1E-7)
fa = FlowAccumulator(grid, runoff_rate='surface_water__specific_discharge')


In [14]:
#Advance timestep. Default units are meters and seconds, though the component is unit agnostic.
dt = 1E3
for i in range(1000):
    gdp.run_one_step(dt)

#calculate curface water flux out of domain
fa.run_one_step()

np.testing.assert_almost_equal(gdp.calc_sw_flux_out(),0.0005077)

